In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Between Human Dignity and Security: Identifying Citizen and Elite Preferences and Concerns over Refugee Reception

---



## **Loading data & modules**



In [ ]:
# Importing libraries and modules
import os
import pandas as pd
import plotly.graph_objects as go

from src.text_normalizations import (
    abbreviation_creator,
    bigram_topic_matrix_creator,
    normalize_text_for_topic_analysis,
    text_normalizer,
    topic_dictionary,
    topic_cleaner,
    unify_citizens_councilors_texts,
    unigram_topic_matrix_creator,
)
import json

In [ ]:
# Read config json
config_path = "config.json"
with open(config_path, "r") as config:
    config_dict = json.load(config)

# Assign paths to variables
citizens_full_path = config_dict["citizens full excel path"]
councilors_full_path = config_dict["councilors full excel path"]
text_analysis_path = config_dict["text analysis data excel path"]
councilors_spelled_path = config_dict["councilors spelled excel path"]
stopwords_path = config_dict["stopwords excel path"]

In [ ]:
# Loading datasets and stopwords
citizens_full = pd.read_excel(citizens_full_path)
councilors_full = pd.read_excel(councilors_full_path)
text_analysis = pd.read_excel(text_analysis_path)
text_analysis.dropna(inplace=True)
text_analysis = pd.merge(
    left=text_analysis,
    right=citizens_full,
    left_on="Anonymous_id",
    right_on="Anonymous_id",
)
text_analysis.dropna(subset=["Q30_campfeedback"], inplace=True)
councilors_spelled = pd.read_excel(councilors_spelled_path)
councilors_spelled.dropna(subset=["Q30"], inplace=True)
councilors_spelled = pd.merge(
    left=councilors_spelled,
    right=councilors_full,
    left_on="id_anonymous",
    right_on="id_anonymous",
)
councilors_spelled.dropna(subset=["Q30"], inplace=True)
stop_words_df = pd.read_excel(stopwords_path, sheet_name="stopwords")

In [ ]:
# Apply text normalization
text_analysis["cleaned"] = text_analysis.Q30_campfeedback.apply(
    lambda x: text_normalizer(x)
)
councilors_spelled["cleaned"] = councilors_spelled.Q30.apply(
    lambda x: text_normalizer(x)
)

# Extract abbreviations
text_analysis["cleaned"] = text_analysis.cleaned.apply(
    lambda x: abbreviation_creator(x)
)
councilors_spelled["cleaned"] = councilors_spelled.cleaned.apply(
    lambda x: abbreviation_creator(x)
)

In [ ]:
stem_dict_1, stem_dict_2 = unify_citizens_councilors_texts(
    citizens_df=text_analysis, councilors_df=councilors_spelled
)

## Topic Analysis

In [ ]:
text_analysis, councilors_spelled = normalize_text_for_topic_analysis(
    text_analysis=text_analysis,
    councilors_spelled=councilors_spelled,
    stem_dict_1=stem_dict_1,
    stem_dict_2=stem_dict_2,
)

In [ ]:
# Drop nan values from both citizens and councilors
citizens = text_analysis.dropna(subset=["cleaned"])
councilors = councilors_spelled.dropna(subset=["cleaned"])

old_cols = [
    "Identity Characteristics",
    "Legal rationale",
    "Cultural/ Social  concerns",
    "Public order concerns",
    "Economic concerns",
    "Humanitarian concerns",
    "Mobility concerns",
    "Trust in authorities",
    "Fairness",
]

new_cols = [
    "Identity",
    "Legal",
    "Cultural/Social",
    "Public Order",
    "Economic",
    "Humanitarian",
    "Mobility",
    "Trust Authorities",
    "Fairness",
]

citizens = citizens.rename(
    columns={
        col: new_col
        for col, new_col in zip(
            old_cols,
            new_cols,
        )
    }
)

councilors = councilors.rename(
    columns={col: new_col for col, new_col in zip(old_cols, new_cols)}
)

In [ ]:
# Read topics' unigrams and bigrams
topics_path = config_dict["topic excel path"]

unigrams = pd.read_excel(topics_path, sheet_name="unigrams")
bigrams = pd.read_excel(topics_path, sheet_name="bigrams")

### Figure 7: Topic analysis on citizens and councilors

In [ ]:
# Apply topic cleaner on each dataframe
citizens.cleaned = citizens.cleaned.apply(lambda x: topic_cleaner(x))
councilors.cleaned = councilors.cleaned.apply(lambda x: topic_cleaner(x))

In [ ]:
# Drop column from both dataframes
unigrams.drop(1, inplace=True)
bigrams.drop(1, inplace=True)

# Extract unigrams and bigrams
unigrams = unigrams.iloc[:,1:]
bigrams = bigrams.iloc[:,1:]

In [ ]:
# Fill nan values with the string 'nothing'
unigrams.fillna("nothing", inplace=True)
bigrams.fillna("nothing", inplace=True)

In [ ]:
# Create unigrams and bigrams dictionaries
unigram_dict = topic_dictionary(unigrams) 
bigram_dict = topic_dictionary(bigrams)

In [ ]:
# Create unigram and bigram matrices for both citizens and councilors
citizens_unigrams_matrix = unigram_topic_matrix_creator(
    dataframe=citizens, topic_dic=unigram_dict
)
citizens_bigrams_matrix = bigram_topic_matrix_creator(
    dataframe=citizens, topic_dic=bigram_dict
)

councilors_unigrams_matrix = unigram_topic_matrix_creator(
    dataframe=councilors, topic_dic=unigram_dict
)
councilors_bigrams_matrix = bigram_topic_matrix_creator(
    dataframe=councilors, topic_dic=bigram_dict
)

In [ ]:
# Add unigrams and bigrams matrices to the associated matrices
citizens_matrix, councilors_matrix = (
    citizens_unigrams_matrix + citizens_bigrams_matrix,
    councilors_unigrams_matrix + councilors_bigrams_matrix,
)

# Iterate through citizens matrix
for row in range(citizens_matrix.shape[0]):
    for column in range(citizens_matrix.shape[1]):
        # If element is greater than 0, assign to it the value 1
        if citizens_matrix[row, column] > 0:
            citizens_matrix[row, column] = 1

# Iterate through citizens matrix
for row in range(councilors_matrix.shape[0]):
    for column in range(councilors_matrix.shape[1]):
        # If element is greater than 0, assign to it the value 1
        if councilors_matrix[row, column] > 0:
            councilors_matrix[row, column] = 1

In [ ]:
# Concatenate matrices to then original dataframes
citizens_df = pd.concat([citizens, pd.DataFrame(citizens_matrix,
             columns=new_cols)], axis=1)

# Concatenate matrices to then original dataframes
councilors_df = pd.concat([councilors, pd.DataFrame(councilors_matrix,
             columns=new_cols)], axis=1)

In [ ]:
# If political orientation is greater than 4, assign "Δεξιοί" else "Αριστεροί"
citizens_df.pol_orient = citizens_df.pol_orient.apply(
    lambda x: "Δεξιοί" if x > 4 else "Αριστεροί"
)
councilors_df.pol_orient_x = councilors_df.pol_orient_x.apply(
    lambda x: "Δεξιοί" if x > 4 else "Αριστεροί"
)

In [ ]:
# Group each dataframe by political orientation column
citizens_pol_orient_df = citizens_df.groupby("pol_orient")[new_cols].sum()

councilors_pol_orient_df = councilors_df.groupby("pol_orient_x")[new_cols].sum()

# Apply feature engineering by creating a column "Total" representing the sum of topics values
citizens_pol_orient_df.loc["Total"] = citizens_pol_orient_df.sum()
councilors_pol_orient_df.loc["Total"] = councilors_pol_orient_df.sum()

In [ ]:
import numpy as np

# Create dataframe where the columns are topic names and rows are the associated topic occurrences
citizens_topics = pd.DataFrame(
    dict(citizens_df[new_cols].sum()), index=["Total"]
).T.reset_index()

# Divide "Total" column with the sum of the topics in order to get topic frequencies
citizens_topics.Total = citizens_topics.Total.apply(
    lambda x: x / (citizens_df[citizens_df.iloc[:, -9:].sum(axis=1) != 0].shape[0])
)

# Calculate statistics for each column
summary = citizens_df[new_cols].describe().T
summary["mean"] = summary["mean"]
summary["std"] = summary["std"]
summary["n"] = len(citizens_df)

# Compute confidence intervals
confidence_level = 0.95
z_score = 1.96  # Z-score for 95% CI
summary["margin_of_error"] = z_score * (summary["std"] / np.sqrt(summary["n"]))
summary["lower_ci"] = summary["mean"] - summary["margin_of_error"]
summary["upper_ci"] = summary["mean"] + summary["margin_of_error"]

# Create a DataFrame for plotting
plot_data = pd.DataFrame(
    {
        "Column": summary.index,
        "Mean": summary["mean"],
        "Lower CI": summary["lower_ci"],
        "Upper CI": summary["upper_ci"],
        "Margin of Error": summary["margin_of_error"],
    }
)

citizens_topics["Mean"] = plot_data["Mean"].values.tolist()
citizens_topics["Lower CI"] = plot_data["Lower CI"].values.tolist()
citizens_topics["Upper CI"] = plot_data["Upper CI"].values.tolist()
citizens_topics["Margin of Error"] = plot_data["Margin of Error"].values.tolist()

In [ ]:
# Create dataframe where the columns are topic names and rows are the associated topic occurrences
councilors_topics = pd.DataFrame(
    dict(councilors_df[new_cols].sum()), index=["Total"]
).T.reset_index()

# Divide "Total" column with the sum of the topics in order to get topic frequencies
councilors_topics.Total = councilors_topics.Total.apply(
    lambda x: x / (councilors_df[councilors_df.iloc[:, -9:].sum(axis=1) != 0].shape[0])
)

# Calculate statistics for each column
summary = councilors_df[new_cols].describe().T
summary["mean"] = summary["mean"]
summary["std"] = summary["std"]
summary["n"] = len(councilors_df)

# Compute confidence intervals
confidence_level = 0.95
z_score = 1.96  # Z-score for 95% CI
summary["margin_of_error"] = z_score * (summary["std"] / np.sqrt(summary["n"]))
summary["lower_ci"] = summary["mean"] - summary["margin_of_error"]
summary["upper_ci"] = summary["mean"] + summary["margin_of_error"]

# Create a DataFrame for plotting
plot_data = pd.DataFrame(
    {
        "Column": summary.index,
        "Mean": summary["mean"],
        "Lower CI": summary["lower_ci"],
        "Upper CI": summary["upper_ci"],
    }
)

councilors_topics["Mean"] = plot_data["Mean"].values.tolist()
councilors_topics["Lower CI"] = plot_data["Lower CI"].values.tolist()
councilors_topics["Upper CI"] = plot_data["Upper CI"].values.tolist()

In [ ]:
# Extract concerns and append them to a list for citizens and councilors
citizens_concerns = citizens_topics["index"].values.tolist()
councilors_concernss = councilors_topics["index"].values.tolist()

# Create figure
fig = go.Figure()

# Add labels
fig.add_trace(
    go.Bar(
        name="Citizens",
        x=citizens_concerns,
        y=citizens_topics.Total,
        textposition="outside",
        marker_color="crimson",
    )
)

# Add labels
fig.add_trace(
    go.Bar(
        name="Councilors",
        x=councilors_concernss,
        y=councilors_topics.Total,
        textposition="outside",
        marker_color="pink",
        marker_pattern_shape="",
    )
)


# Change the bar mode
fig.update_layout(
    title_x=0.5, barmode="group", template="plotly_white", font=dict(size=22)
)

fig.update_layout(
    annotations=[
        dict(
            text="Frequency of mentions",
            x=-0.05,  # Adjust the position as needed
            y=1.05,  # Position above the plot (1.05 places it above the plot area)
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(size=22, color="black"),  # Font size and color
            xanchor="left",  # Anchor the text to the left
            yanchor="bottom",  # Anchor the text at the bottom
        )
    ]
)

fig.update_xaxes(tickfont=dict(size=22))

fig.write_html(
    file="../figures/Figure_7.html",
)

fig.show()

In [ ]:
# Copy citizes_pol_orient_df
citizens_topics = citizens_pol_orient_df.copy()

# Get topic frequencies
citizens_topics.iloc[0, :] = (
    citizens_topics.iloc[0, :]
    / citizens_df[
        (citizens_df.pol_orient == "Αριστεροί")
        & (citizens_df.iloc[:, -9:].sum(axis=1) != 0)
    ].shape[0]
)
citizens_topics.iloc[1, :] = (
    citizens_topics.iloc[1, :]
    / citizens_df[
        (citizens_df.pol_orient == "Δεξιοί")
        & (citizens_df.iloc[:, -9:].sum(axis=1) != 0)
    ].shape[0]
)

# Copy councilors_pol_orient_df
councilors_topics = councilors_pol_orient_df.copy()

# Get topic frequencies
councilors_topics.iloc[0, :] = (
    councilors_topics.iloc[0, :]
    / councilors_df[
        (councilors_df.pol_orient_x == "Αριστεροί")
        & (councilors_df.iloc[:, -9:].sum(axis=1) != 0)
    ].shape[0]
)
councilors_topics.iloc[1, :] = (
    councilors_topics.iloc[1, :]
    / councilors_df[
        (councilors_df.pol_orient_x == "Δεξιοί")
        & (councilors_df.iloc[:, -9:].sum(axis=1) != 0)
    ].shape[0]
)

### Figure 8: Topic analysis on citizens and councilors by their political orientation

In [ ]:
# Extract concerns and append them to a list for citizens and councilors
citizens_concerns = citizens_topics.columns.tolist()
councilors_concerns = councilors_topics.columns.tolist()

# Create figure
fig = go.Figure()

# Add labels
fig.add_trace(
    go.Bar(
        name="Left-wing citizens",
        x=citizens_concerns,
        y=citizens_topics.reset_index().iloc[0, 1:],
        textposition="outside",
        marker_color="crimson",
    )
)

# Add labels
fig.add_trace(
    go.Bar(
        name="Left-wing councilors",
        x=councilors_concerns,
        y=councilors_topics.reset_index().iloc[0, 1:],
        textposition="outside",
        marker_color="pink",
        marker_pattern_shape="",
    )
)

# Add labels
fig.add_trace(
    go.Bar(
        name="Right-wing citizens",
        x=citizens_concerns,
        y=citizens_topics.reset_index().iloc[1, 1:],
        textposition="outside",
        marker_color="royalblue",
    )
)

# Add labels
fig.add_trace(
    go.Bar(
        name="Right-wing councilors",
        x=councilors_concerns,
        y=councilors_topics.reset_index().iloc[1, 1:],
        textposition="outside",
        marker_color="lightblue",
        marker_pattern_shape="",
    )
)


# Change the bar mode
fig.update_layout(
    title_x=0.5, barmode="group", template="plotly_white", font=dict(size=22)
)

fig.update_layout(
    annotations=[
        dict(
            text="Frequency of mentions",
            x=-0.05,  # Adjust the position as needed
            y=1.05,  # Position above the plot (1.05 places it above the plot area)
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(size=22, color="black"),  # Font size and color
            xanchor="left",  # Anchor the text to the left
            yanchor="bottom",  # Anchor the text at the bottom
        )
    ]
)

fig.update_xaxes(tickfont=dict(size=22))

fig.write_html(
    file="../figures/Figure_8.html",
)

# Show figure
fig.show()

In [ ]:
# Read updated dataset containing columns 'exposed' and 'intensity'
imm_pop_path = config_dict["immigrants populations excel path"]
imm_pop = pd.read_excel(imm_pop_path)

# Map exposed, intensity and political orientation values
exposed = {0: "Non exposed", 1: "Exposed"}
intensity = {1: "Not exposed", 2: "Low exposure", 3: "High exposure"}

imm_pop.exposed = imm_pop.exposed.apply(lambda x: exposed[x])
imm_pop.intensity = imm_pop.intensity.apply(lambda x: intensity[x])
imm_pop.pol_orient = imm_pop.pol_orient.apply(
    lambda x: "Right-wing" if x > 4 else "Left-wing"
)

In [ ]:
# Concatenate matrices to then original dataframes
imm_pop_df = pd.concat(
    [imm_pop, pd.DataFrame(citizens_matrix, columns=new_cols)], axis=1
)

In [ ]:
# Group each dataframe by exposure column
imm_pop_exposure = imm_pop_df.groupby("exposed")[new_cols].sum()

# Apply feature engineering by creating a column "Total" representing the sum of topics values
imm_pop_exposure.loc["Total"] = imm_pop_exposure.sum()

In [ ]:
# Get citizens topics
citizens_topics = imm_pop_exposure.copy()

# Get topic frequencies
citizens_topics.iloc[0, :] = (
    citizens_topics.iloc[0, :]
    / imm_pop_df[
        (imm_pop_df.exposed == "Exposed") & (imm_pop_df.iloc[:, -9:].sum(axis=1) != 0)
    ].shape[0]
)
citizens_topics.iloc[1, :] = (
    citizens_topics.iloc[1, :]
    / imm_pop_df[
        (imm_pop_df.exposed == "Non exposed")
        & (imm_pop_df.iloc[:, -9:].sum(axis=1) != 0)
    ].shape[0]
)

In [ ]:
# Read dataset
imm_pop = pd.read_excel(imm_pop_path)

# Map exposure, intensity and political orientation
imm_pop.pol_orient = imm_pop.pol_orient.apply(
    lambda x: "Right-wing" if x > 4 else "Left-wing"
)
imm_pop.exposed = imm_pop.exposed.apply(lambda x: {0: "No camp", 1: "Camp"}[x])
imm_pop.loc[
    imm_pop.Q26_residence_2.isin(
        ["Mytilinis", "Anatolikis Samou", "Lerou", "Chiou", "Ko", "Orestiadas"]
    ),
    "exposed",
] = "RIC"

In [ ]:
# Concatenate matrices to then original dataframes
imm_pop_df = pd.concat(
    [imm_pop, pd.DataFrame(citizens_matrix, columns=new_cols)], axis=1
)

In [ ]:
# Group each dataframe by exposure column
imm_pop_exposure = imm_pop_df.groupby("exposed")[new_cols].sum()

# Apply feature engineering by creating a column "Total" representing the sum of topics values
imm_pop_exposure.loc["Total"] = imm_pop_exposure.sum()

In [ ]:
# Get citizens topics
citizens_topics = imm_pop_exposure.copy()

# Get topic frequencies
citizens_topics.iloc[0, :] = (
    citizens_topics.iloc[0, :]
    / imm_pop_df[
        (imm_pop_df.exposed == "Camp") & (imm_pop_df.iloc[:, -9:].sum(axis=1) != 0)
    ].shape[0]
)
citizens_topics.iloc[1, :] = (
    citizens_topics.iloc[1, :]
    / imm_pop_df[
        (imm_pop_df.exposed == "No camp") & (imm_pop_df.iloc[:, -9:].sum(axis=1) != 0)
    ].shape[0]
)
citizens_topics.iloc[2, :] = (
    citizens_topics.iloc[2, :]
    / imm_pop_df[
        (imm_pop_df.exposed == "RIC") & (imm_pop_df.iloc[:, -9:].sum(axis=1) != 0)
    ].shape[0]
)

### Figure 12: Exposure intensity, citizens

In [ ]:
# Get concerns
citizens_concerns = citizens_topics.columns.tolist()

# Add figure
fig = go.Figure()

# Add labels
fig.add_trace(
    go.Bar(
        name="No camp",
        x=citizens_concerns,
        y=citizens_topics.reset_index().iloc[1, 1:],
        textposition="outside",
        marker_color="blue",
    )
)

fig.add_trace(
    go.Bar(
        name="Camp",
        x=citizens_concerns,
        y=citizens_topics.reset_index().iloc[0, 1:],
        textposition="outside",
        marker_color="pink",
    )
)

fig.add_trace(
    go.Bar(
        name="RIC",
        x=citizens_concerns,
        y=citizens_topics.reset_index().iloc[2, 1:],
        textposition="outside",
        marker_color="red",
    )
)

# Change the bar mode
fig.update_layout(
    title_x=0.5, barmode="group", template="plotly_white", font=dict(size=22)
)

fig.update_layout(
    annotations=[
        dict(
            text="Frequency of mentions",
            x=-0.05,  # Adjust the position as needed
            y=1.05,  # Position above the plot (1.05 places it above the plot area)
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(size=22, color="black"),  # Font size and color
            xanchor="left",  # Anchor the text to the left
            yanchor="bottom",  # Anchor the text at the bottom
        )
    ]
)

fig.update_xaxes(tickfont=dict(size=22))

fig.write_html(
    file="../figures/Figure_12.html",
)

fig.show()

In [ ]:
# Read dataset
imm_pop = pd.read_excel(imm_pop_path)

# Map exposure, intensity and political orientation
imm_pop.pol_orient = imm_pop.pol_orient.apply(
    lambda x: "Right-wing" if x > 4 else "Left-wing"
)
imm_pop.exposed = imm_pop.exposed.apply(lambda x: {0: "No camp", 1: "Camp"}[x])
imm_pop.loc[
    imm_pop.Q26_residence_2.isin(
        ["Mytilinis", "Anatolikis Samou", "Lerou", "Chiou", "Ko", "Orestiadas"]
    ),
    "exposed",
] = "RIC"

In [ ]:
# Create a list containing citizens grouped by their political orientation and exposure
imm_pol_exposure = []

for i in range(len(imm_pop)):
    political_orient = imm_pop.pol_orient.iloc[i]
    exposure = imm_pop.exposed.iloc[i]
    outcome = political_orient + " | " + exposure
    imm_pol_exposure.append(outcome)

imm_pop["pol and exposed"] = imm_pol_exposure

In [ ]:
# Concatenate matrices to then original dataframes
imm_pop_df = pd.concat(
    [imm_pop, pd.DataFrame(citizens_matrix, columns=new_cols)], axis=1
)

In [ ]:
# Group each dataframe by exposure column
imm_pop_exposure = imm_pop_df.groupby("pol and exposed")[new_cols].sum()

# Apply feature engineering by creating a column "Total" representing the sum of topics values
imm_pop_exposure.loc["Total"] = imm_pop_exposure.sum()

In [ ]:
# Get citizens topics
citizens_topics = imm_pop_exposure.copy()

# Get topic frequencies
citizens_topics.iloc[0, :] = (
    citizens_topics.iloc[0, :]
    / imm_pop_df[
        (imm_pop_df["pol and exposed"] == "Left-wing | Camp")
        & (imm_pop_df.iloc[:, -9:].sum(axis=1) != 0)
    ].shape[0]
)
citizens_topics.iloc[1, :] = (
    citizens_topics.iloc[1, :]
    / imm_pop_df[
        (imm_pop_df["pol and exposed"] == "Left-wing | No camp")
        & (imm_pop_df.iloc[:, -9:].sum(axis=1) != 0)
    ].shape[0]
)
citizens_topics.iloc[2, :] = (
    citizens_topics.iloc[2, :]
    / imm_pop_df[
        (imm_pop_df["pol and exposed"] == "Left-wing | RIC")
        & (imm_pop_df.iloc[:, -9:].sum(axis=1) != 0)
    ].shape[0]
)
citizens_topics.iloc[3, :] = (
    citizens_topics.iloc[3, :]
    / imm_pop_df[
        (imm_pop_df["pol and exposed"] == "Right-wing | Camp")
        & (imm_pop_df.iloc[:, -9:].sum(axis=1) != 0)
    ].shape[0]
)
citizens_topics.iloc[4, :] = (
    citizens_topics.iloc[4, :]
    / imm_pop_df[
        (imm_pop_df["pol and exposed"] == "Right-wing | No camp")
        & (imm_pop_df.iloc[:, -9:].sum(axis=1) != 0)
    ].shape[0]
)
citizens_topics.iloc[5, :] = (
    citizens_topics.iloc[5, :]
    / imm_pop_df[
        (imm_pop_df["pol and exposed"] == "Right-wing | RIC")
        & (imm_pop_df.iloc[:, -9:].sum(axis=1) != 0)
    ].shape[0]
)

### Figure 13: Exposure intensity by political orientation, citizens

In [ ]:
# Get concerns
citizens_concerns = citizens_topics.columns.tolist()

# Add figure
fig = go.Figure()

# Add labels
fig.add_trace(
    go.Bar(
        name="Left-wing | No camp",
        x=citizens_concerns,
        y=citizens_topics.reset_index().iloc[1, 1:],
        textposition="outside",
        marker_color="yellow",
    )
)

fig.add_trace(
    go.Bar(
        name="Right-wing | No camp",
        x=citizens_concerns,
        y=citizens_topics.reset_index().iloc[4, 1:],
        textposition="outside",
        marker_color="green",
    )
)

fig.add_trace(
    go.Bar(
        name="Left-wing | Camp",
        x=citizens_concerns,
        y=citizens_topics.reset_index().iloc[0, 1:],
        textposition="outside",
        marker_color="pink",
    )
)

fig.add_trace(
    go.Bar(
        name="Right-wing | Camp",
        x=citizens_concerns,
        y=citizens_topics.reset_index().iloc[3, 1:],
        textposition="outside",
        marker_color="cyan",
    )
)

fig.add_trace(
    go.Bar(
        name="Left-wing | RIC",
        x=citizens_concerns,
        y=citizens_topics.reset_index().iloc[2, 1:],
        textposition="outside",
        marker_color="red",
    )
)

fig.add_trace(
    go.Bar(
        name="Right-wing | RIC",
        x=citizens_concerns,
        y=citizens_topics.reset_index().iloc[5, 1:],
        textposition="outside",
        marker_color="blue",
    )
)

# Change the bar mode
fig.update_layout(
    title_x=0.5, barmode="group", template="plotly_white", font=dict(size=22)
)

fig.update_layout(
    annotations=[
        dict(
            text="Frequency of mentions",
            x=-0.05,  # Adjust the position as needed
            y=1.05,  # Position above the plot (1.05 places it above the plot area)
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(size=22, color="black"),  # Font size and color
            xanchor="left",  # Anchor the text to the left
            yanchor="bottom",  # Anchor the text at the bottom
        )
    ]
)

fig.update_xaxes(tickfont=dict(size=22))

fig.write_html(
    file="../figures/Figure_13.html",
)

fig.show()